# Adaptive Cards Toolkit: 05a - Confirmation Templates

This notebook demonstrates how to create confirmation dialog templates for user actions and decision-making. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook.

In [ ]:
# First run the common utilities notebook
%run 01_toolkit_common.ipynb

# Print methods available to help us understand the API
show_available_methods()

## Learning Objectives

In this notebook, you'll learn how to:

1. Create basic confirmation dialogs
2. Add detailed information to confirmations
3. Style confirmation actions based on risk level
4. Build multi-step confirmation workflows
5. Create different types of confirmation templates for various scenarios

## Introduction to Confirmation Templates

Confirmation templates are used to confirm user actions, especially for destructive, irreversible, or significant operations. They typically include:

- A clear title that indicates confirmation is required
- A message explaining what action will be taken
- Details about the item or action being confirmed
- Confirm and cancel buttons with appropriate styling

Good confirmation design:
- Clearly explains the consequences of the action
- Makes the destructive action visually distinct
- Provides enough context for an informed decision
- Offers a clear way to cancel or go back

## Basic Confirmation Template

Let's create a function that generates a basic confirmation dialog.

In [ ]:
# Create a confirmation template function
def create_confirmation(title, message, confirm_button_text="Confirm", cancel_button_text="Cancel", details=None):
    """Create a confirmation card for user actions.
    
    Args:
        title: Confirmation title
        message: Confirmation message
        confirm_button_text: Text for the confirm button
        cancel_button_text: Text for the cancel button
        details: Optional list of detail strings
        
    Returns:
        A confirmation card
    """
    # Create the base card
    card = builder.create_basic_card(
        title="",  # We'll add our own title with styling
        message=""
    )
    
    # Create warning header
    warning_container = LayoutHelper.create_container(
        items=[
            ElementFactory.create_heading("⚠️ CONFIRMATION REQUIRED", level=2)
        ],
        style="warning"
    )
    
    # Add header to card
    card.body.append(warning_container)
    
    # Add title and message
    card.body.append(ElementFactory.create_heading(title, level=2))
    card.body.append(ElementFactory.create_text(message))
    
    # Add details if provided
    if details and len(details) > 0:
        details_items = [ElementFactory.create_heading("Details:", level=3)]
        
        for detail in details:
            details_items.append(ElementFactory.create_text(detail))
        
        details_container = LayoutHelper.create_container(
            items=details_items,
            style="emphasis"
        )
        
        card.body.append(details_container)
    
    # Add confirmation and cancel actions
    confirm_action = {"type": "Action.Submit", "title": confirm_button_text, "data": {"action": "confirm"}, "style": "destructive"}
    cancel_action = {"type": "Action.Submit", "title": cancel_button_text, "data": {"action": "cancel"}}
    
    card.actions = [confirm_action, cancel_action]
    
    return card

## Basic Confirmation Example

In [ ]:
# Create a basic confirmation dialog
confirm_card = create_confirmation(
    title="Confirm Deletion",
    message="Are you sure you want to delete this item? This action cannot be undone.",
    confirm_button_text="Yes, Delete",
    cancel_button_text="Cancel"
)

display_card(confirm_card, "Confirmation Dialog")

## Confirmation with Details

Confirmations often need to include details about what's being affected to help users make informed decisions.

In [ ]:
# Create a confirmation with additional details
detailed_confirm = create_confirmation(
    title="Confirm Order Cancellation",
    message="Are you sure you want to cancel this order? This will release inventory and initiate a refund process.",
    details=[
        "Order #: ORD-2024-15678",
        "Items: 3",
        "Total: $245.97",
        "Status: Processing"
    ],
    confirm_button_text="Yes, Cancel Order",
    cancel_button_text="Keep Order"
)

display_card(detailed_confirm, "Detailed Confirmation")

## Risk-Based Confirmation

Different actions have different levels of risk or impact. Let's create confirmations that are styled based on the risk level.

In [ ]:
# Create a risk-based confirmation template function
def create_risk_confirmation(title, message, risk_level="medium", confirm_button_text="Confirm", cancel_button_text="Cancel", details=None):
    """Create a confirmation card styled based on risk level.
    
    Args:
        title: Confirmation title
        message: Confirmation message
        risk_level: Risk level (low, medium, high)
        confirm_button_text: Text for the confirm button
        cancel_button_text: Text for the cancel button
        details: Optional list of detail strings
        
    Returns:
        A risk-based confirmation card
    """
    # Create the base card
    card = builder.create_basic_card(
        title="",  # We'll add our own title with styling
        message=""
    )
    
    # Set styling based on risk level
    if risk_level == "low":
        icon = "ℹ️"
        style = "emphasis"
        header_text = "CONFIRMATION"
        button_style = None  # No special style for low risk
    elif risk_level == "medium":
        icon = "⚠️"
        style = "warning"
        header_text = "CONFIRMATION REQUIRED"
        button_style = "destructive"
    elif risk_level == "high":
        icon = "🛑"
        style = "attention"
        header_text = "CRITICAL ACTION CONFIRMATION"
        button_style = "destructive"
    else:  # Default to medium
        icon = "⚠️"
        style = "warning"
        header_text = "CONFIRMATION REQUIRED"
        button_style = "destructive"
    
    # Create header container
    warning_container = LayoutHelper.create_container(
        items=[
            ElementFactory.create_heading(f"{icon} {header_text}", level=2)
        ],
        style=style
    )
    
    # Add header to card
    card.body.append(warning_container)
    
    # Add title and message
    card.body.append(ElementFactory.create_heading(title, level=2))
    card.body.append(ElementFactory.create_text(message))
    
    # Add details if provided
    if details and len(details) > 0:
        details_items = [ElementFactory.create_heading("Details:", level=3)]
        
        for detail in details:
            details_items.append(ElementFactory.create_text(detail))
        
        details_container = LayoutHelper.create_container(
            items=details_items,
            style="emphasis"
        )
        
        card.body.append(details_container)
    
    # For high risk, add an explicit checkbox for additional confirmation
    if risk_level == "high":
        # Add a confirmation input with a checkbox
        try:
            # Try to add a choice set for confirmation
            card.body.append(ElementFactory.create_text("Please confirm you understand the consequences:", weight="bolder"))
            
            confirmation_choice = ElementFactory.create_choice_set(
                id="explicit_confirmation",
                choices=[
                    {"title": "I understand this action cannot be undone", "value": "confirmed"}
                ],
                is_required=True,
                style="expanded"  # Checkbox style
            )
            
            card.body.append(confirmation_choice)
        except Exception as e:
            # If choice set creation fails, add a simple text note
            card.body.append(ElementFactory.create_text("This action has significant consequences and cannot be undone.", weight="bolder"))
    
    # Add confirmation and cancel actions
    confirm_action = {"type": "Action.Submit", "title": confirm_button_text, "data": {"action": "confirm", "risk_level": risk_level}}
    
    # Add style to confirm button for medium and high risk
    if button_style:
        confirm_action["style"] = button_style
    
    cancel_action = {"type": "Action.Submit", "title": cancel_button_text, "data": {"action": "cancel"}}
    
    # For high risk, put the cancel button first to reduce accidental confirmation
    if risk_level == "high":
        card.actions = [cancel_action, confirm_action]
    else:
        card.actions = [confirm_action, cancel_action]
    
    return card

## Risk Level Examples

In [ ]:
# Create a low-risk confirmation
low_risk = create_risk_confirmation(
    title="Mark Message as Read",
    message="Do you want to mark all messages as read?",
    risk_level="low",
    confirm_button_text="Yes, Mark as Read",
    cancel_button_text="Cancel"
)

display_card(low_risk, "Low Risk Confirmation")

# Create a medium-risk confirmation
medium_risk = create_risk_confirmation(
    title="Archive Project",
    message="Are you sure you want to archive this project? It will be moved to the archive and removed from active projects.",
    risk_level="medium",
    details=[
        "Project: Website Redesign",
        "Status: Completed",
        "Last activity: 45 days ago",
        "Team members: 5"
    ],
    confirm_button_text="Archive Project",
    cancel_button_text="Keep Active"
)

display_card(medium_risk, "Medium Risk Confirmation")

# Create a high-risk confirmation
high_risk = create_risk_confirmation(
    title="Delete User Account",
    message="WARNING: You are about to permanently delete this user account and all associated data. This action cannot be undone and will result in permanent data loss.",
    risk_level="high",
    details=[
        "User: john.smith@example.com",
        "Account type: Administrator",
        "Account age: 3 years, 2 months",
        "Data to be deleted: Profile information, Posts (245), Comments (1,024), Project assignments (12)"
    ],
    confirm_button_text="Permanently Delete Account",
    cancel_button_text="Cancel Deletion"
)

display_card(high_risk, "High Risk Confirmation")

## Confirmation with Alternatives

Sometimes users need more options than just "confirm" or "cancel". Let's create a confirmation with alternative actions.

In [ ]:
# Create a confirmation with alternative actions
def create_multi_option_confirmation(title, message, options, details=None):
    """Create a confirmation with multiple action options.
    
    Args:
        title: Confirmation title
        message: Confirmation message
        options: List of option objects with title, action, data, and style
        details: Optional list of detail strings
        
    Returns:
        A confirmation card with multiple options
    """
    # Create the base card
    card = builder.create_basic_card(
        title="",  # We'll add our own title with styling
        message=""
    )
    
    # Create header container
    header_container = LayoutHelper.create_container(
        items=[
            ElementFactory.create_heading("⚠️ ACTION REQUIRED", level=2)
        ],
        style="warning"
    )
    
    # Add header to card
    card.body.append(header_container)
    
    # Add title and message
    card.body.append(ElementFactory.create_heading(title, level=2))
    card.body.append(ElementFactory.create_text(message))
    
    # Add details if provided
    if details and len(details) > 0:
        details_items = [ElementFactory.create_heading("Details:", level=3)]
        
        for detail in details:
            details_items.append(ElementFactory.create_text(detail))
        
        details_container = LayoutHelper.create_container(
            items=details_items,
            style="emphasis"
        )
        
        card.body.append(details_container)
    
    # Add options as actions
    actions = []
    for option in options:
        action = {
            "type": "Action.Submit",
            "title": option["title"],
            "data": {"action": option["action"]}
        }
        
        # Add additional data if provided
        if "data" in option and option["data"]:
            for key, value in option["data"].items():
                action["data"][key] = value
        
        # Add style if provided
        if "style" in option and option["style"]:
            action["style"] = option["style"]
        
        actions.append(action)
    
    card.actions = actions
    
    return card

In [ ]:
# Create a confirmation with multiple options
session_expiry = create_multi_option_confirmation(
    title="Session About to Expire",
    message="Your session will expire in 2 minutes due to inactivity. What would you like to do?",
    options=[
        {
            "title": "Continue Session",
            "action": "extend_session",
            "data": {"extend_minutes": 30},
            "style": None
        },
        {
            "title": "Save Work & Logout",
            "action": "save_logout",
            "data": {"save_draft": True},
            "style": None
        },
        {
            "title": "Logout Now",
            "action": "logout",
            "data": None,
            "style": "destructive"
        }
    ]
)

display_card(session_expiry, "Multi-Option Confirmation")

# Create a document conflict resolution dialog
document_conflict = create_multi_option_confirmation(
    title="Document Conflict Detected",
    message="This document has been modified by another user since you opened it. How would you like to proceed?",
    details=[
        "Document: Project Proposal.docx",
        "Your version: Modified 10 minutes ago",
        "Server version: Modified 3 minutes ago by Jane Smith",
        "Changes: 2 paragraphs added, 1 image replaced"
    ],
    options=[
        {
            "title": "Save as New Version",
            "action": "save_new",
            "data": {"document_id": "doc-1234"},
            "style": None
        },
        {
            "title": "View Differences",
            "action": "view_diff",
            "data": {"document_id": "doc-1234"},
            "style": None
        },
        {
            "title": "Overwrite Server Version",
            "action": "overwrite",
            "data": {"document_id": "doc-1234"},
            "style": "destructive"
        },
        {
            "title": "Discard My Changes",
            "action": "discard",
            "data": {"document_id": "doc-1234"},
            "style": "destructive"
        }
    ]
)

display_card(document_conflict, "Document Conflict Resolution")

## Multi-step Confirmation Workflow

For very high-risk actions, a multi-step confirmation process can be used to ensure the user fully understands the consequences and is making a deliberate choice.

In [ ]:
# Create a multi-step confirmation workflow
def create_multistep_confirmation(steps):
    """Create a series of confirmation cards for a multi-step confirmation process.
    
    Args:
        steps: List of step objects with title, message, details, and options
        
    Returns:
        List of confirmation cards for the multi-step process
    """
    cards = []
    
    for i, step in enumerate(steps):
        # Create a header that shows the step number
        step_title = f"Step {i+1} of {len(steps)}: {step['title']}"
        
        # Create options, adding step data
        options = []
        for option in step["options"]:
            opt_copy = option.copy()
            
            # Initialize data dictionary if not present
            if "data" not in opt_copy or not opt_copy["data"]:
                opt_copy["data"] = {}
            else:
                # Create a copy of the data dictionary
                opt_copy["data"] = opt_copy["data"].copy()
            
            # Add step information to data
            opt_copy["data"]["step"] = i + 1
            opt_copy["data"]["total_steps"] = len(steps)
            
            options.append(opt_copy)
        
        # Create the confirmation card for this step
        card = create_multi_option_confirmation(
            title=step_title,
            message=step["message"],
            options=options,
            details=step.get("details")
        )
        
        cards.append(card)
    
    return cards

In [ ]:
# Define a multi-step confirmation workflow for deleting a project
delete_project_steps = [
    {
        "title": "Confirm Project Deletion",
        "message": "You are about to delete the 'Customer Portal Redesign' project. This will permanently remove the project and all its contents.",
        "details": [
            "Project ID: PRJ-2024-056",
            "Created: January 15, 2024",
            "Status: Active",
            "Team members: 8",
            "Files: 156",
            "Tasks: 43 (12 open, 31 completed)"
        ],
        "options": [
            {
                "title": "Continue to Verification",
                "action": "next_step",
                "data": {"project_id": "PRJ-2024-056"},
                "style": None
            },
            {
                "title": "Cancel Deletion",
                "action": "cancel",
                "data": None,
                "style": None
            }
        ]
    },
    {
        "title": "Verify Understanding",
        "message": "Please verify that you understand the consequences of deleting this project:",
        "details": [
            "All project files will be permanently deleted",
            "All task assignments and comments will be lost",
            "Team members will lose access to all project resources",
            "This action cannot be undone",
            "Deletion will occur immediately upon confirmation"
        ],
        "options": [
            {
                "title": "I Understand, Continue",
                "action": "next_step",
                "data": {"project_id": "PRJ-2024-056", "understood": True},
                "style": None
            },
            {
                "title": "Go Back",
                "action": "previous_step",
                "data": None,
                "style": None
            },
            {
                "title": "Cancel Deletion",
                "action": "cancel",
                "data": None,
                "style": None
            }
        ]
    },
    {
        "title": "Final Confirmation",
        "message": "This is your final chance to cancel. Once confirmed, the project will be permanently deleted.",
        "details": [
            "Project: Customer Portal Redesign (PRJ-2024-056)",
            "Administrator authorization is required",
            "This action will be logged for audit purposes"
        ],
        "options": [
            {
                "title": "Cancel Deletion",
                "action": "cancel",
                "data": None,
                "style": None
            },
            {
                "title": "Go Back",
                "action": "previous_step",
                "data": None,
                "style": None
            },
            {
                "title": "Permanently Delete Project",
                "action": "confirm_delete",
                "data": {"project_id": "PRJ-2024-056", "final_confirmation": True},
                "style": "destructive"
            }
        ]
    }
]

# Create the multi-step confirmation cards
delete_project_cards = create_multistep_confirmation(delete_project_steps)

# Display each step
for i, card in enumerate(delete_project_cards):
    display_card(card, f"Project Deletion - Step {i+1} of {len(delete_project_cards)}")

## Contextual Confirmation

Let's create confirmation templates that adapt based on the context of what's being confirmed.

In [ ]:
# Create a contextual confirmation based on the item type
def create_contextual_confirmation(item_type, item_id, item_name=None, additional_details=None):
    """Create a confirmation that's tailored to the type of item being affected.
    
    Args:
        item_type: Type of item (document, user, project, etc.)
        item_id: ID of the item
        item_name: Optional name of the item
        additional_details: Optional additional details
        
    Returns:
        A confirmation card tailored to the item type
    """
    # Set confirmation details based on item type
    if item_type == "document":
        title = f"Confirm Document Deletion"
        message = f"Are you sure you want to delete {item_name or 'this document'}? This action cannot be undone."
        risk_level = "medium"
        confirm_text = "Delete Document"
        details = [
            f"Document ID: {item_id}",
            f"Name: {item_name or 'Unknown'}"
        ]
        
    elif item_type == "user":
        title = f"Confirm User Deactivation"
        message = f"Are you sure you want to deactivate the account for {item_name or 'this user'}? They will lose access to all resources."
        risk_level = "high"
        confirm_text = "Deactivate User"
        details = [
            f"User ID: {item_id}",
            f"Name: {item_name or 'Unknown'}"
        ]
        
    elif item_type == "project":
        title = f"Confirm Project Archival"
        message = f"Are you sure you want to archive {item_name or 'this project'}? It will be moved to the archive and removed from active projects."
        risk_level = "medium"
        confirm_text = "Archive Project"
        details = [
            f"Project ID: {item_id}",
            f"Name: {item_name or 'Unknown'}"
        ]
        
    elif item_type == "payment":
        title = f"Confirm Payment"
        message = f"You are about to make a payment of {item_name or '$0.00'}. This will be charged to your default payment method."
        risk_level = "medium"
        confirm_text = "Confirm Payment"
        details = [
            f"Payment ID: {item_id}",
            f"Amount: {item_name or '$0.00'}"
        ]
        
    elif item_type == "subscription":
        title = f"Confirm Subscription Cancellation"
        message = f"Are you sure you want to cancel your {item_name or 'subscription'}? You will lose access at the end of the current billing period."
        risk_level = "medium"
        confirm_text = "Cancel Subscription"
        details = [
            f"Subscription ID: {item_id}",
            f"Plan: {item_name or 'Unknown'}"
        ]
        
    else:  # Generic confirmation
        title = f"Confirm Action"
        message = f"Are you sure you want to proceed with this action?"
        risk_level = "medium"
        confirm_text = "Confirm"
        details = [
            f"Item ID: {item_id}",
            f"Item Name: {item_name or 'Unknown'}"
        ]
    
    # Add any additional details
    if additional_details:
        details.extend(additional_details)
    
    # Create the confirmation card
    return create_risk_confirmation(
        title=title,
        message=message,
        risk_level=risk_level,
        confirm_button_text=confirm_text,
        cancel_button_text="Cancel",
        details=details
    )

In [ ]:
# Create contextual confirmations for different item types
document_confirm = create_contextual_confirmation(
    item_type="document",
    item_id="DOC-2024-789",
    item_name="Q1 Financial Report.xlsx",
    additional_details=[
        "Size: 2.4 MB",
        "Last modified: February 15, 2024",
        "Modified by: Jane Smith"
    ]
)

display_card(document_confirm, "Document Deletion Confirmation")

user_confirm = create_contextual_confirmation(
    item_type="user",
    item_id="USR-1005",
    item_name="Robert Johnson",
    additional_details=[
        "Email: robert.johnson@example.com",
        "Role: Project Manager",
        "Department: Marketing",
        "Active projects: 3"
    ]
)

display_card(user_confirm, "User Deactivation Confirmation")

payment_confirm = create_contextual_confirmation(
    item_type="payment",
    item_id="PMT-20240227-003",
    item_name="$1,299.00",
    additional_details=[
        "Payment method: Visa ending in 4242",
        "Billing address: 123 Main St, Anytown, USA",
        "Items: Premium Plan Annual Subscription",
        "Next billing date: February 27, 2025"
    ]
)

display_card(payment_confirm, "Payment Confirmation")

## Confirmation Template Library

Let's organize our confirmation templates into a reusable library class.

In [ ]:
# Create a confirmation template library class
class ConfirmationTemplates:
    """A library of confirmation dialog templates."""
    
    def __init__(self, builder):
        """Initialize with a card builder."""
        self.builder = builder
    
    def create_basic_confirmation(self, title, message, confirm_button_text="Confirm", cancel_button_text="Cancel", details=None):
        """Create a basic confirmation dialog."""
        return create_confirmation(title, message, confirm_button_text, cancel_button_text, details)
    
    def create_risk_confirmation(self, title, message, risk_level="medium", confirm_button_text="Confirm", cancel_button_text="Cancel", details=None):
        """Create a risk-based confirmation dialog."""
        return create_risk_confirmation(title, message, risk_level, confirm_button_text, cancel_button_text, details)
    
    def create_multi_option_confirmation(self, title, message, options, details=None):
        """Create a confirmation with multiple action options."""
        return create_multi_option_confirmation(title, message, options, details)
    
    def create_multistep_confirmation(self, steps):
        """Create a multi-step confirmation workflow."""
        return create_multistep_confirmation(steps)
    
    def create_contextual_confirmation(self, item_type, item_id, item_name=None, additional_details=None):
        """Create a confirmation based on the item type."""
        return create_contextual_confirmation(item_type, item_id, item_name, additional_details)

# Initialize the template library
confirmation_templates = ConfirmationTemplates(builder)

# Use the template library
library_confirmation = confirmation_templates.create_contextual_confirmation(
    item_type="subscription",
    item_id="SUB-1234",
    item_name="Premium Plan",
    additional_details=[
        "Current billing cycle: $29.99/month",
        "Active since: November 15, 2023",
        "Benefits you'll lose: Priority support, Advanced analytics, Custom integrations"
    ]
)

display_card(library_confirmation, "Confirmation Library Example")

## Key Takeaways

In this notebook, you've learned:

1. How to create basic confirmation dialogs for user actions
2. How to add detailed information to help users make informed decisions
3. How to style confirmations based on risk level to communicate severity
4. How to build multi-option confirmations for complex decisions
5. How to implement multi-step confirmation workflows for high-risk actions
6. How to create contextual confirmations that adapt to different scenarios
7. How to organize confirmation templates into a reusable library

In the next notebook, we'll explore content templates for displaying articles and information.

## Exercises

Try these exercises to reinforce what you've learned:

1. Create a timed confirmation template that automatically cancels after a specified time period
2. Modify the multi-step confirmation workflow to include a password verification step for high-security actions
3. Create a "soft confirmation" template that uses checkboxes for multiple confirmations in a single dialog